# Introduction

`mock` is a library that helps you write better tests.

It was created as an independent python library: https://github.com/testing-cabal/mock

It is now part of the Python standard library, available as `unittest.mock` in Python3.3 onwards.
In Python2.7, you have to `pip install mock`. Many (if not all) of the features that are only available in Python3.5 or 3.6 are available in the latest version of independent `mock` library.

# `mock.Mock` object

A `Mock` object creates, by default, all attributes and methods as you access them and stores details on how they have been used. You can configure them, specify its return values  or limit what attributes are available, and make assertions about how they have been used.

Full documentation: https://docs.python.org/3/library/unittest.mock.html

In [23]:
from unittest.mock import Mock
animal = Mock()
animal.potato = 'potato attribute'

print(animal.potato)
print(animal.something)
print(animal.method)
print(animal.method())

animal.method.assert_called_once()
animal.othermethod.assert_not_called()

print('Method called {} times'.format(animal.method.call_count))

potato attribute
<Mock name='mock.something' id='140455162428944'>
<Mock name='mock.method' id='140455162429112'>
<Mock name='mock.method()' id='140455162429336'>
Method called 1 times


# `mock.MagicMock` object
Subclass of `Mock` with default implementations of most of the magic methods.

Unlike `Mock`, you can do:

In [2]:
from unittest.mock import MagicMock
m = MagicMock()
print(m.__int__(), m.__len__(), m.__bool__())
print('x' in m)
print([x for x in m])
print(list(m))
with m as context:
    print('`with` returns {}'.format(context))


1 0 True
False
[]
[]
`with` returns <MagicMock name='mock.__enter__()' id='140455163565000'>


However, the following magic methods are also implemented in `Mock` objects:

In [3]:
from unittest.mock import Mock
m = Mock()
print(m.__hash__())
print(m.__str__())
print(m.__repr__())
print(m.__sizeof__())

-9223363258406502151
<Mock id='140455172378520'>
<Mock id='140455172378520'>
32


## How mock objects are created as they are being accessed

The default behaviour of a normal `Mock()` object is the following:
- *Calling* a `Mock` creates a new Mock. It always returns the same mock, regardless of its parameters.

In [4]:
mymock = Mock(name='main_mock')
print(mymock)
a = mymock(34)
b = mymock(3, potato=False)
assert a is b
print(a)
print(b)

<Mock name='main_mock' id='140455163453392'>
<Mock name='main_mock()' id='140455163450312'>
<Mock name='main_mock()' id='140455163450312'>


- First access on attribute creates a new `Mock`

In [5]:
mymock = Mock()
print(mymock.a)
print(mymock.a)

<Mock name='mock.a' id='140455172377064'>
<Mock name='mock.a' id='140455172377064'>


- Some attributes/methods are reserved by Mock class and are initialized when Mock object is created:
    - `name`
    - `return_value`
    - `assert_called_once`
    - `assert_called_once_with`
    - `call_count`
    - etc...
- Index accesses are not supported by default
- Magic methods are not supported by default -> use MagicMock instead. Explained above

### Example
In the following example, we are mocking a real python method.

In [6]:
SQL_INSERT = "INSERT INTO table (name, potato) VALUES (%(name)s, %(potato)s)"
def insert_row(connection, row):
    # Function which inserts a row in a database
    cursor = connection.cursor()
    cursor.execute(SQL_INSERT, row)
    connection.commit()

connection_mock = Mock()
sample_row = {'name': 'ABC', 'potato': 123}
insert_row(connection_mock, sample_row)

connection_mock.cursor.assert_called_once()
connection_mock.cursor().execute.assert_called_once_with(SQL_INSERT, sample_row)
connection_mock.commit.assert_called_once()

Note that, as explained above, `connection_mock.cursor()` will always return the same `Mock` object. Same for all `connection` and `cursor` attributes.

# `Mock.__init__` parameters

### return_value and side_effect
It is possible to define which value should be returned when calling a `Mock` with `return_value` attribute.

In [7]:
m = Mock(return_value=123)
m.method.return_value = 'wow'
assert m() == 123
assert m.method() == 'wow'

The `side_effect` attribute let's you define the function which will be called when the mock is called.

In [8]:
def myfunction(x):
    return x + 2
m = Mock(side_effect=myfunction)
m.other = lambda: 'potato'
assert m(4) == 6
assert m.other() == 'potato'

m.assert_called_once()
try:
    m.other.assert_called_once()
except:
    print('m.other is not a Mock object!')

m.other is not a Mock object!


`side_effect` can also have other uses, depending on what you assign to it.

If you set an `Exception` object as a `side_effect`, calling the mock will raise this exception:

In [9]:
m = Mock()
m.side_effect = Exception('Function raises an exception')
try:
    m()
except Exception as e:
    print(e)

Function raises an exception


If you set an `iterable`, it will be used as an order of returned values

In [10]:
from unittest.mock import DEFAULT
m = Mock()
m.side_effect = [1, DEFAULT, 2, DEFAULT, Exception('oops'), 3]
m.return_value = 'the return_value'

for call_number in range(8):
    try:
        result = m()
        print('Call {} - result => {}'.format(call_number, result))
    except Exception as e:
        print('Call {} - {} {}'.format(call_number, type(e), e))

m.side_effect = None  # Unset side_effect makes it return the return_value
print('Last Call -  result =', m())

Call 0 - result => 1
Call 1 - result => the return_value
Call 2 - result => 2
Call 3 - result => the return_value
Call 4 - <class 'Exception'> oops
Call 5 - result => 3
Call 6 - <class 'StopIteration'> 
Call 7 - <class 'StopIteration'> 
Last Call -  result = the return_value


### `name` attribute
Used to have a more meaningful `repr` of the mock. Useful for debugging.
Name is propagated to its child mocks.

In [11]:
m = Mock(name='abc')
m.other = Mock()
print(m)
print(m())
print(m.method())
print(m.other().very.deep)

<Mock name='abc' id='140455163196696'>
<Mock name='abc()' id='140455163196304'>
<Mock name='abc.method()' id='140455163196024'>
<Mock name='abc.other().very.deep' id='140455172378128'>


### `unsafe` attribute and typos
(New in Python 3.5)

By default if any attribute starts with `assert` or `assret` will raise an AttributeError, *very useful to not write typos when attempting to assert something*. You can disable this behaviour by passing `unsafe=True` to `Mock(...)`:

In [24]:
# Using unsafe=True
m = Mock(unsafe=True)
m(4)
m.assert_called_one_time(4)
print('1 - Assertions ok!')

# Default (unsafe=False)
m = Mock()
m(4)
try:
    m.assert_called_one_time(4)
    print('2 - Assertions ok!')
except AttributeError as e:
    print('2 - {}'.format(repr(e)))

# unsafe is not propagated to child mocks!
m = Mock(unsafe=True)
m.method(5)
m.assert_called_potato(5)
m.method.assert_called_potato(5)
print('3 - All good!')

1 - Assertions ok!
2 - AttributeError('assert_called_one_time',)


AttributeError: assert_called_potato

### `wraps` attribute



In [13]:
from unittest.mock import Mock, DEFAULT
def myfunction(a, b):
    return a + b

m = Mock(wraps=myfunction)
print('result=', m(1, 2))
m.assert_called_once_with(1, 2)

result= 3


#### `wraps` vs `return_value` and `side_effect`
`return_value` has priority over the `wraps` function.

In [14]:
m.return_value = 'custom return value'
print('result=', m(1, 2))

result= custom return value


You can undefine a `return_value` by assigning it to `DEFAULT`

In [15]:
m.return_value = DEFAULT
print('result=', m(1, 2))

result= 3


`side_effect` does not override `wraps`, but still calls it!

In [16]:
def otherfunction(a, b):
    print('*** called otherfunction', a, b)
    return 'other result'

m = Mock(wraps=myfunction, side_effect=otherfunction)
print('result=', m(1, 2))

*** called otherfunction 1 2
result= 3


However, `side_effect` overrides `return_value`...

In [17]:
m = Mock(wraps=myfunction, side_effect=otherfunction)
m.return_value = 'myreturn'
print('result=', m(1, 2))

*** called otherfunction 1 2
result= other result


Summary:
- `side_effect` is always called/consumed when defined
- mock returns first of:
    - `return_value` (if `return_value != DEFAULT`)
    - `wraps` (if `wraps != None`)
    - `side_effect` result

### `spec` attribute
`spec` can be a list of strings or an existing object (class or instance) that acts as specification for the mock object. Mock will initialize all its methods/attributes with this specification. Accessing any attribute not in the spec will raise an AttributeError.

In [18]:
class Animal:
    def __init__(self, name):
        self.name = name
    
    def get_name():
        return name
    
    def is_animal():
        return True
    
    def method(self, a, b, c):
        return b

m = Mock(spec=Animal)
print(m)
assert hasattr(m, 'method')
assert not hasattr(m, 'other')
assert isinstance(m, Animal)
assert isinstance(m.method(), Mock)
m.method.assert_called_once()

m = Mock(spec=['potato'])
print(m.potato)
assert hasattr(m, 'potato')
assert not hasattr(m, 'other')


<Mock spec='Animal' id='140455163197872'>
<Mock name='mock.potato' id='140455163196136'>


**Creating an instance from a mocked class object will not return an instance with the same `spec`!!**

In [19]:
class_mock = Mock(spec=Animal)
print('Class:', class_mock.__class__)
print('get_name' in dir(class_mock))

pepe_instance_mock = class_mock('Pepe')
print('Class:', pepe_instance_mock.__class__)
print('get_name' in dir(pepe_instance_mock))

Class: <class '__main__.Animal'>
True
Class: <class 'unittest.mock.Mock'>
False


**What can happen if you don't use `spec`:**

In [20]:
class Person:
    def __init__(self):
        self.name = name
    
    def get_name(self):
        if self.naem:
            self.write('Retrieved non-empty name')
        return self.name

class_mock = MagicMock()
tom = class_mock('Tom ')

class_mock.asert_called_with('Tom')

Person.get_name(tom)
tom.write.assert_called_with('Retrieved non-empty name')

Using `spec`:

In [21]:
class_mock = MagicMock(spec=Person)
tom = class_mock('Tom ')
class_mock.asert_called_with('Tom')

AttributeError: Mock object has no attribute 'asert_called_with'

The missing `self.naem` attribute and `self.write` method will not be raised because `tom` does not have the `spec` of `Animal`, therefore a `TypeError` will not be raised.

# History of mock calls
Every mock tracks all the calls made to itself and all its children.

List of attributes:
- `called (bool)`: `True` if mock was called at least once.
- `call_count (int)`: How many times the mock has been called.
- `call_args`: None if never called. Otherwise, a `mock.call` object of the last call
- `call_args_list`: List of all calls to the mock as `mock.call` objects.
- `method_calls`: As well as tracking calls to themselves, mocks also track calls to its childs calls. List of `mock.call` objects too.
- `mock_calls`: Like `method_calls`, but also includes calls to itself, the magic methods (`__int__`, `__str__`, etc...) and all its child `mock_calls`.

The list of available `assert_` methods are:
- `assert_called`: Asserts mock was **called at least once**
- `assert_called_with`: Asserts mock's **last call was done with specific parameters**
- `assert_any_call`: Asserts was **called at least once with specific parameters** (minimum one parameter).
- `assert_has_calls(calls, any_order=False)`: Asserts the mock has been **called with the specified calls** (list of `mock.call`), in a specific order or any.

Other methods:
- `reset_mock`: Resets all call attributes on a mock object


**New in Python3.5:**
- `assert_not_called`: Assert the mock was never called

**New in Python3.6:**
- `assert_called_once`: Asserts mock was **called exactly once**
- `assert_called_once_with`: Asserts was **called exactly once with specific parameters**

These assert functions already exist in latest version of non-builtin `mock` (>=1.1.0) library, which is the library you use with Python2.7.


In [25]:
def show_mock_stats(message, mock):
    print('{:-^64}'.format(message))
    for attr in ['called', 'call_count', 'call_args', 'call_args_list', 'method_calls']:
        print('{:>20}: {}'.format(attr, getattr(mock, attr)))
    print()

m = Mock()
show_mock_stats('Empty mock:', m)

m()
m(5, 6)
m.method(1, 2, 3)
m.potato.size = 5
m.climb.mountain(water=True)
show_mock_stats('Mock with calls:', m)

m.reset_mock()
show_mock_stats('After reset', m)

--------------------------Empty mock:---------------------------
              called: False
          call_count: 0
           call_args: None
      call_args_list: []
        method_calls: []

------------------------Mock with calls:------------------------
              called: True
          call_count: 2
           call_args: call(5, 6)
      call_args_list: [call(), call(5, 6)]
        method_calls: [call.method(1, 2, 3), call.climb.mountain(water=True)]

--------------------------After reset---------------------------
              called: False
          call_count: 0
           call_args: None
      call_args_list: []
        method_calls: []

